In [ ]:
# Setup: Ensure the animal_shelter module is in the Python path
import sys
from pathlib import Path

candidate = Path.cwd()
project_root = None
for _ in range(6):
    if (candidate / "animal_shelter").is_dir():
        project_root = candidate
        break
    candidate = candidate.parent

if project_root and str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")

In [ ]:
# Imports
from dash import Dash, dcc, html
from dash import dash_table
from dash.dependencies import Input, Output
from dash import dcc as dash_dcc  # alias for optional ECharts integration
try:
    from dash_echarts import DashECharts
except Exception:
    DashECharts = None
import dash_leaflet as dl
import pandas as pd
import base64
import os

# CRUD module
from animal_shelter.animal_shelter import AnimalShelter

In [ ]:
# Model: Connect to MongoDB and fetch all records
AAC_USER = "aacuser"
AAC_PASS = "SECRET"

shelter = AnimalShelter(user=AAC_USER, password=AAC_PASS)
# Ensure indexes for performance
try:
    shelter.ensure_indexes()
except Exception:
    pass

# Detect coordinate columns using a single sample document
lat_col = None
lon_col = None
sample = shelter.collection.find_one({}, {"location_lat": 1, "lat": 1, "latitude": 1, "y": 1, "location_long": 1, "lon": 1, "longitude": 1, "x": 1}) or {}
for c in ["location_lat", "lat", "latitude", "y"]:
    if c in sample:
        lat_col = c
        break
for c in ["location_long", "lon", "longitude", "x"]:
    if c in sample:
        lon_col = c
        break
if lat_col is None:
    lat_col = "location_lat"
if lon_col is None:
    lon_col = "location_long"

# Default visible columns
visible_columns = [
    'animal_id', 'name', 'animal_type', 'breed', 'age_upon_outcome', 'outcome_type', 'sex_upon_outcome', 'datetime'
]
# Append coordinates for map (hidden in UI)
table_columns = [c for c in visible_columns] + [lat_col, lon_col]

# Projection for DB queries (performance)
projection_fields = {col: 1 for col in table_columns}
MAX_ROWS = int(os.getenv('MAX_ROWS', '2000'))

# Build initial data directly from MongoDB with projection and limit (avoid full DataFrame)
all_records_data = []
for doc in shelter.collection.find({}, projection_fields).limit(MAX_ROWS):
    doc.pop('_id', None)
    for k, v in list(doc.items()):
        doc[k] = "" if v is None else v
    all_records_data.append(doc)
print(f"Records fetched (limited): {len(all_records_data)}")

# Precompute breed/category helpers
def rescue_category_criteria(category: str) -> dict:
    if category in ('Water Rescue',):
        return {
            "$and": [
                {"animal_type": "Dog"},
                {"breed": {"$regex": "Labrador|Chesapeake Bay Retriever|Newfoundland", "$options": "i"}},
                {"sex_upon_outcome": {"$regex": "Intact", "$options": "i"}}
            ]
        }
    if category in ('Mountain Rescue', 'Mountain or Wilderness Rescue'):
        return {
            "$and": [
                {"animal_type": "Dog"},
                {"breed": {"$regex": "German Shepherd|Old English Sheepdog|Siberian Husky|Rottweiler|Doberman", "$options": "i"}},
                {"sex_upon_outcome": {"$regex": "Intact", "$options": "i"}}
            ]
        }
    if category in ('Disaster Rescue', 'Disaster or Individual Tracking'):
        return {
            "$and": [
                {"animal_type": "Dog"},
                {"breed": {"$regex": "German Shepherd|Doberman|Rottweiler|Bloodhound", "$options": "i"}},
                {"sex_upon_outcome": {"$regex": "Intact", "$options": "i"}}
            ]
        }
    return {}

unique_id = "Dave Mobley - Project Two"

# Cache for filtered results
FILTER_CACHE = {"Reset": all_records_data}

def get_filtered_records(filter_type: str) -> list:
    if not filter_type or filter_type == 'Reset':
        return FILTER_CACHE.get('Reset', all_records_data)
    if filter_type in FILTER_CACHE:
        return FILTER_CACHE[filter_type]
    criteria = rescue_category_criteria(filter_type)
    try:
        cursor = shelter.collection.find(criteria, projection_fields).limit(MAX_ROWS)
        docs = []
        for doc in cursor:
            doc.pop('_id', None)
            for k, v in list(doc.items()):
                doc[k] = "" if v is None else v
            docs.append(doc)
        FILTER_CACHE[filter_type] = docs
        return docs
    except Exception:
        return all_records_data

In [ ]:
# View: Build the dashboard layout
external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']
app = Dash(__name__, external_stylesheets=external_stylesheets, assets_folder=str(project_root / 'assets'))

# Logo image
logo_path = project_root / 'Grazioso Salvare Logo.png'
logo_b64 = None
if logo_path.exists():
    with open(logo_path, 'rb') as f:
        logo_b64 = base64.b64encode(f.read()).decode('ascii')

filter_options = [
    {'label': 'Reset', 'value': 'Reset'},
    {'label': 'Water Rescue', 'value': 'Water Rescue'},
    {'label': 'Mountain or Wilderness Rescue', 'value': 'Mountain or Wilderness Rescue'},
    {'label': 'Disaster or Individual Tracking', 'value': 'Disaster or Individual Tracking'}
]

chart_options = [
    {'label': 'Bar', 'value': 'bar'},
    {'label': 'Pie', 'value': 'pie'},
    {'label': 'Treemap', 'value': 'treemap'}
]

app.layout = html.Div(
    className='container',
    children=[
        dcc.Store(id='filtered-data', data=all_records_data),
        html.Div([
            html.A(
                html.Img(src=f"data:image/png;base64,{logo_b64}" if logo_b64 else '', style={'height': '60px'}),
                href='https://www.snhu.edu',
                target='_blank',
                rel='noopener noreferrer',
                style={'display': 'inline-block'}
            )
        ], style={'textAlign': 'center', 'marginTop': '16px'}),
        html.H1('SNHU CS-340 Dashboard - Project Two', style={'textAlign': 'center'}),
        html.P(unique_id, style={'textAlign': 'center'}),
        html.Hr(),

        html.Div([
            html.Label('Rescue Category'),
            dcc.RadioItems(
                id='filter-type',
                options=filter_options,
                value='Reset',
                inline=True
            ),
            html.Div(style={'height': '8px'}),
            html.Label('Chart Type'),
            dcc.Dropdown(id='chart-type', options=chart_options, value='bar', clearable=False, style={'width': '260px'})
        ], style={'marginBottom': '12px'}),

        html.Div(id='table-container', children=[
            dcc.Loading(type='circle', children=[
                dash_table.DataTable(
                    id='datatable-id',
                    columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in table_columns],
                    data=all_records_data,
                    page_action='native',
                    page_current=0,
                    page_size=25,
                    sort_action='native',
                    filter_action='none',
                    row_selectable='single',
                    selected_rows=[0],
                    fixed_rows={'headers': True},
                    virtualization=True,
                    fill_width=False,
                    style_table={'overflowX': 'auto', 'overflowY': 'auto', 'height': '55vh', 'width': '100%'},
                    style_cell={
                        'minWidth': '100px', 'width': '140px', 'maxWidth': '280px',
                        'whiteSpace': 'nowrap',
                        'textOverflow': 'ellipsis',
                        'overflow': 'hidden',
                        'padding': '8px',
                        'fontFamily': '-apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Helvetica, Arial, sans-serif',
                        'fontSize': 13, 'lineHeight': '18px',
                        'border': '1px solid #eaeaea'
                    },
                    style_header={
                        'backgroundColor': '#2a5bd7',
                        'color': 'white',
                        'fontWeight': 'bold',
                        'border': '0px'
                    },
                    style_cell_conditional=[
                        {'if': {'column_id': lat_col}, 'display': 'none'},
                        {'if': {'column_id': lon_col}, 'display': 'none'},
                    ],
                    style_as_list_view=True,
                )
            ])
        ]),

        html.Br(),
        html.Hr(),
        html.Div(className='row', style={'display': 'flex', 'gap': '16px'}, children=[
            html.Div(id='graph-id', className='col s12 m6', style={'flex': '1'}),
            html.Div(id='map-id', className='col s12 m6', style={'flex': '1'})
        ])
    ]
)

In [ ]:
# Controller: callbacks for filter, table highlighting, charts, and map

# Filter: update table data based on category
@app.callback(
    [Output('datatable-id','data'), Output('filtered-data','data')],
    [Input('filter-type', 'value')]
)
def update_dashboard(filter_type):
    try:
        if not filter_type or filter_type == 'Reset':
            return all_records_data, all_records_data
        records_local = get_filtered_records(filter_type)
        return records_local, records_local
    except Exception:
        return all_records_data, all_records_data

# Highlight selected columns and rows
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns'),
     Input('datatable-id', 'selected_rows')]
)
def update_styles(selected_columns, selected_rows):
    selected_columns = selected_columns or []
    selected_rows = selected_rows or []
    col_styles = [{ 'if': {'column_id': col}, 'background_color': '#E8F2FF' } for col in selected_columns]
    row_styles = [{ 'if': {'row_index': idx}, 'background_color': '#FFF6E5' } for idx in selected_rows]
    return col_styles + row_styles

# Chart: preferred breeds chart from current table with animated transitions and chart type selection
@app.callback(
    Output('graph-id', 'children'),
    [Input('filtered-data', 'data'), Input('chart-type', 'value')]
)
def update_graphs(data, chart_type):
    try:
        if DashECharts is None:
            return [html.Div("ECharts not available. Please install dash-echarts.")]
        # Build DataFrame from current filtered data
        dff = pd.DataFrame.from_records(data) if data else pd.DataFrame(all_records_data)
        if dff.empty or 'breed' not in dff.columns:
            return []
        top = dff['breed'].astype(str).value_counts().nlargest(10).reset_index()
        top.columns = ['breed', 'count']
        # Create ECharts option based on chart type
        top = top.sort_values(['count', 'breed'], ascending=[False, True])
        labels = top['breed'].tolist()
        values = top['count'].tolist()
        if chart_type == 'pie':
            option = {
                'title': {'text': 'Top Breeds (Current Selection)', 'left': 'left', 'textStyle': {'fontSize': 16}},
                'tooltip': {'trigger': 'item', 'formatter': '{b}: {d}% ({c})'},
                'legend': {'orient': 'vertical', 'left': 'right'},
                'series': [{
                    'name': 'Breeds', 'type': 'pie', 'radius': ['35%', '60%'], 'avoidLabelOverlap': True,
                    'itemStyle': {'borderRadius': 3, 'borderColor': '#fff', 'borderWidth': 1},
                    'label': {'show': True, 'formatter': '{b}\n{d}%'},
                    'data': [{'name': l, 'value': v} for l, v in zip(labels, values)]
                }]
            }
        elif chart_type == 'treemap':
            option = {
                'title': {'text': 'Top Breeds (Current Selection)', 'left': 'left', 'textStyle': {'fontSize': 16}},
                'tooltip': {'formatter': '{b}: {c}'},
                'series': [{
                    'type': 'treemap', 'label': {'show': True, 'formatter': '{b}'},
                    'breadcrumb': {'show': False},
                    'data': [{'name': l, 'value': int(v)} for l, v in zip(labels, values)]
                }]
            }
        else:
            option = {
                'title': {'text': 'Top Breeds (Current Selection)', 'left': 'left', 'textStyle': {'fontSize': 16}},
                'tooltip': {'trigger': 'axis'},
                'xAxis': {'type': 'category', 'data': labels, 'axisLabel': {'rotate': 0}},
                'yAxis': {'type': 'value', 'name': 'Count', 'splitLine': {'lineStyle': {'opacity': 0.15}}},
                'series': [{'type': 'bar', 'data': values, 'itemStyle': {'borderRadius': [3,3,0,0]}}]
            }
        return [DashECharts(option=option, style={'width': '100%', 'height': '420px'})]
    except Exception:
        return []

# Map: update based on current selection, with layers control and clustering
@app.callback(
    Output('map-id', 'children'),
    [Input('datatable-id', 'derived_virtual_data'),
     Input('datatable-id', 'derived_virtual_selected_rows')]
)
def update_map(viewData, index):
    dff = pd.DataFrame.from_records(viewData) if viewData is not None else pd.DataFrame(all_records_data)
    if dff.empty:
        return html.Div("No data available for map.")

    # Collect marker positions (limit for performance)
    positions = []
    markers = []
    max_markers = 500
    if lat_col in dff.columns and lon_col in dff.columns:
        for _, rec in dff[[lat_col, lon_col, 'name', 'breed', 'animal_type', 'outcome_type']].head(max_markers).iterrows():
            try:
                if pd.notna(rec[lat_col]) and pd.notna(rec[lon_col]):
                    pos = [float(rec[lat_col]), float(rec[lon_col])]
                    positions.append(pos)
                    popup = dl.Popup([
                        html.Div([
                            html.H4(str(rec.get('name', 'Unknown')), style={'margin': '0 0 6px 0'}),
                            html.Div(str(rec.get('animal_type', 'Animal')), style={'fontSize': '14px', 'color': '#555'}),
                            html.Div(str(rec.get('breed', 'Unknown breed')), style={'fontSize': '14px', 'color': '#555'}),
                            html.Div(str(rec.get('outcome_type', '') or ''), style={'fontSize': '13px', 'color': '#777'})
                        ])
                    ])
                    markers.append(dl.Marker(position=pos, children=[dl.Tooltip(str(rec.get('breed', ''))), popup]))
            except Exception:
                continue

    # Determine selected row and focus
    row = 0 if not index else index[0]
    row = max(0, min(row, len(dff) - 1))
    selected_center = None
    try:
        lat = dff.iloc[row].get(lat_col)
        lon = dff.iloc[row].get(lon_col)
        if pd.notna(lat) and pd.notna(lon):
            selected_center = [float(lat), float(lon)]
    except Exception:
        selected_center = None

    # Build layers control with basemaps and overlay
    base_osm = dl.BaseLayer(dl.TileLayer(id='base-osm'), name='OSM', checked=True)
    base_toner = dl.BaseLayer(dl.TileLayer(id='base-toner', url='https://{s}.tile.stamen.com/toner/{z}/{x}/{y}.png'), name='Toner')
    base_terrain = dl.BaseLayer(dl.TileLayer(id='base-terrain', url='https://{s}.tile.stamen.com/terrain/{z}/{x}/{y}.jpg'), name='Terrain')

    overlay = None
    if hasattr(dl, 'MarkerClusterGroup'):
        overlay = dl.Overlay(dl.MarkerClusterGroup(children=markers), name='Animals', checked=True)
    else:
        overlay = dl.Overlay(dl.LayerGroup(children=markers), name='Animals', checked=True)

    layers = dl.LayersControl(children=[base_osm, base_toner, base_terrain, overlay])

    # Compute bounds if no focused marker
    map_kwargs = {'style': {'width': '100%', 'height': '60vh'}}
    if selected_center is not None:
        map_kwargs.update({'center': selected_center, 'zoom': 14})
    elif positions:
        lats = [p[0] for p in positions]
        lons = [p[1] for p in positions]
        bounds = [[min(lats), min(lons)], [max(lats), max(lons)]]
        map_kwargs.update({'bounds': bounds})
    else:
        map_kwargs.update({'center': [30.75, -97.48], 'zoom': 10})

    return [dl.Map(children=[layers, dl.ScaleControl(position='bottomleft')], **map_kwargs)]

In [ ]:
# Run the app
app.run(jupyter_mode='external')